# World Cup Goals

by Benjamín Meneses in behalf of LenioLabs

In [1]:
import numpy as np
import pandas as pd

In [2]:
players = pd.read_csv("WorldCupPlayers.csv")
matches = pd.read_csv("WorldCupMatches.csv")
world_cup = pd.read_csv("WorldCups.csv")

In [3]:
matches.dropna(inplace=True)

In [4]:
# Names Correction
names = matches[matches['Home Team Name'].str.contains('rn">')]['Home Team Name'].value_counts()
wrong = list(names.index)
correct = [name.split('>')[1] for name in wrong]
old_name = ['Germany FR', 'Maracan� - Est�dio Jornalista M�rio Filho', 'Estadio do Maracana']
new_name = ['Germany', 'Maracan Stadium', 'Maracan Stadium']
wrong = wrong + old_name
correct = correct + new_name

In [5]:
# Replace the wrong names with the correct ones
for index, wr in enumerate(wrong):
    world_cup = world_cup.replace(wrong[index], correct[index])
    
for index, wr in enumerate(wrong):
    matches = matches.replace(wrong[index], correct[index])
    
for index, wr in enumerate(wrong):
    players = players.replace(wrong[index], correct[index])

In [6]:
world_cup

,Year,Country,Winner,Runners-Up,Third,Fourth,GoalsScored,QualifiedTeams,MatchesPlayed,Attendance
0,1930,Uruguay,Uruguay,Argentina,USA,Yugoslavia,70,13,18,590.549
1,1934,Italy,Italy,Czechoslovakia,Germany,Austria,70,16,17,363.000
2,1938,France,Italy,Hungary,Brazil,Sweden,84,15,18,375.700
3,1950,Brazil,Uruguay,Brazil,Sweden,Spain,88,13,22,1.045.246
4,1954,Switzerland,Germany,Hungary,Austria,Uruguay,140,16,26,768.607
5,1958,Sweden,Brazil,Sweden,France,Germany,126,16,35,819.810
6,1962,Chile,Brazil,Czechoslovakia,Chile,Yugoslavia,89,16,32,893.172
7,1966,England,England,Germany,Portugal,Soviet Union,89,16,32,1.563.135
8,1970,Mexico,Brazil,Italy,Germany,Uruguay,95,16,32,1.603.975
9,1974,Germany,Germany,Netherlands,Poland,Brazil,97,16,38,1.865.753


In [7]:
qatarCountries = [
    "Qatar", "Ecuador", "Senegal",
    "Netherlands", "England",
    "Iran", "USA", "Argentina",
    "Saudi Arabia", "Mexico", "Poland",
    "France", "Denmark", "Tunisia",
    "Spain", "Germany", "Japan",
    "Belguim", "Canada",
    "Morocco", "Croatia",
    "Brasil", "Serbia",
    "Switzerland", "Cameroon", "Portugal",
    "Ghana", "Uruguay", "Korea Republic"
]

# Get Goals
filteredMatches = matches[matches['Year'] >= 1998].drop(columns=[
    'Attendance', 'Half-time Home Goals',
    'Half-time Away Goals', 'Referee', 'Assistant 1', 'Assistant 2',
    'RoundID'])
#filteredMatches = filteredMatches[filteredMatches['Home Team Name'].isin(qatarCountries) | filteredMatches['Away Team Name'].isin(qatarCountries)]


In [8]:
import datetime

In [9]:
# Get Player Goals per Match
worldCupGoals = pd.DataFrame()
for idx, match in filteredMatches.iterrows():
    goals = match['Home Team Goals'] + match['Away Team Goals']
    playerGoals = players[players['MatchID'] == match['MatchID']].loc[:, ['Event', 'Player Name', 'Team Initials']].dropna()
    # Keep only the goal events
    playerGoals['isGoal'] = playerGoals['Event'].str.contains("[G,W,P]\d*'")
    cleanPlayerGoals = playerGoals[playerGoals['isGoal'] == True].drop_duplicates()
    # Iterate over the goals and add the goals
    gottenGoals = 0
    for _, goal in cleanPlayerGoals.iterrows():
        eventString = list(filter(lambda x: x.startswith(("G", "W", "P")), goal['Event'].split(" ")))
        for eventKey in eventString:
            minute = int(eventKey[1:-1])
            # Add the goal to the dataframe
            goalRow = pd.concat([match, goal], axis=0)
            goalRow = goalRow.drop(labels=['isGoal', 'Event'])
            try:
                goalRow['Date'] = datetime.datetime.strptime(goalRow['Datetime'], '%d %b %Y - %H:%M ')
            except:
                goalRow['Date'] = datetime.datetime.strptime(goalRow['Datetime'], '%d %B %Y - %H:%M ')
            goalRow['Minute'] = minute
            goalRow['GoalType'] = eventKey[0]
            worldCupGoals = pd.concat([worldCupGoals, pd.DataFrame(goalRow).transpose()])
            gottenGoals += 1
    if(gottenGoals != goals):
        print(gottenGoals, goals)
        print(cleanPlayerGoals)
        print(match)
        raise Exception("Goals and events don't match up")
worldCupGoals.sort_values(by=['Date', 'Minute'], inplace=True)
worldCupGoals.drop_duplicates(inplace=True)
# Reset index and add it as an id
worldCupGoals.reset_index(inplace=True, drop=True)
worldCupGoals['id'] = worldCupGoals.index + 1

In [10]:
worldCupGoals.to_json("worldCupGoals.json", orient='records')